# NPC AI - Phi-3 QLoRA Fine-tuning (Kaggle)

This notebook enables you to fine-tune the `Phi-3-mini` model on Kaggle using a P100 or T4 GPU. It uses `unsloth` for efficient training.

**Pre-requisites:**
1.  **Internet Access**: Enable internet in the notebook settings (right sidebar).
2.  **Accelerator**: Select GPU P100 or T4 x2.
3.  **Input Data**: Attach your `npc_training.jsonl` (and ideally the repo scripts) as an input dataset.

In [ ]:
%%capture
# Install Unsloth for Kaggle
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
# Setup Environment
import os
import shutil

# Copy training data to working directory for write access
# Search for the file in inputs
input_file = None
for root, dirs, files in os.walk("/kaggle/input"):
    if "npc_training.jsonl" in files:
        input_file = os.path.join(root, "npc_training.jsonl")
        break

if input_file:
    print(f"Found input file at: {input_file}")
    shutil.copy(input_file, "/kaggle/working/npc_training.jsonl")
else:
    print("WARNING: npc_training.jsonl not found in inputs! Please upload it.")
    # Fallback: Create dummy file for testing if needed
    # with open("npc_training.jsonl", "w") as f: f.write(...) 


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
# Formatting Data
alpaca_prompt = """
### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token 
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    outputs      = examples["completion"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files = "npc_training.jsonl", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, 
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer.train()

In [ ]:
# Export to GGUF (Available in /kaggle/working/model)
# You can download the output files from the right sidebar in Kaggle
model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")